# Modeling with TensorFlow


In [28]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
import cv2 as cv
from imageai.Detection import ObjectDetection
import os
import random, math

In [14]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    tf.test.gpu_device_name()
    # need to use gpu runtime accelerator!

# Create generator


In [15]:
!pwd

/c/Users/User/Documents/GitHub/sidewalks_deep-learning


In [21]:
os.chdir(f'./images/')

In [22]:
!pwd

/c/Users/User/Documents/GitHub/sidewalks_deep-learning/images


In [29]:
#move images into train, test, validate folders with subfolders for classes
test_ratio = .15
SPLIT_DATA = True # switch if need to divide out data
if SPLIT_DATA:
    for label in ['sidewalk', 'no_sidewalk']:
        %cd ./$label
        listing = os.popen('ls').read().strip().split(sep='\n')
        random.shuffle(listing) #inplace
        im_count = len(listing)
        test_size=val_size = math.ceil(test_ratio*im_count)
        train_size = im_count - (test_size + val_size)
        subfolders = ['train', 'test', 'vdate']
        for subfolder in subfolders:
            os.makedirs(os.path.join(os.path.dirname(os.getcwd()), subfolder, label))
        for item in range(0, train_size):
            moved = listing.pop()
            !mv $moved ../train/$label/$moved
        for item in range(0, test_size):
            moved = listing.pop()
            !mv $moved ../test/$label/$moved
        for item in range(0, val_size):
            moved = listing.pop()
            !mv $moved ../vdate/$label/$moved
        %cd ..
        !rmdir ./$label

[WinError 2] The system cannot find the file specified: './sidewalk'
C:\Users\User\Documents\GitHub\sidewalks_deep-learning\images\sidewalk
C:\Users\User\Documents\GitHub\sidewalks_deep-learning\images
C:\Users\User\Documents\GitHub\sidewalks_deep-learning\images\no_sidewalk


Parameter format not correct - "sidewalk".


C:\Users\User\Documents\GitHub\sidewalks_deep-learning\images


Invalid switch - "no_sidewalk".


### Parameters for data loading and modeling

In [41]:
idg_params={'rescale': 1./255}
augmentation_params = {'rotation_range': 30,
                        'zoom_range': 0.15,
                         'width_shift_range': 0.2,
                         'height_shift_range': 0.2,
                         'shear_range': 0.15,
                         'fill_mode': 'nearest',
                         'horizontal_flip': True}
datagen_params = {'batch_size': 32,
                  'target_size': (256,256),
                  'color_mode': 'rgb', 
                  'class_mode': 'binary'}
batch_size = 128
n_epochs = 15
IMG_HEIGHT = 200
IMG_WIDTH = 200

In [32]:
train_datagen = ImageDataGenerator(**idg_params)
train_augmented_datagen = ImageDataGenerator(
    **idg_params,
    **augmentation_params)
test_datagen = ImageDataGenerator(**idg_params)
vdate_datagen = ImageDataGenerator(**idg_params)

In [33]:
# Flow training images in batches of 128 using train_datagen generator

train_generator = train_datagen.flow_from_directory(
        r'./train/',  # This is the source directory for training images
        **datagen_params)
test_generator = test_datagen.flow_from_directory(
        r'./test/',
        **datagen_params)
vdate_generator = vdate_datagen.flow_from_directory(
        r'./vdate/',
        **datagen_params)

Found 2087 images belonging to 2 classes.
Found 448 images belonging to 2 classes.
Found 448 images belonging to 2 classes.


In [34]:
#generator based caluculations
input_shape = train_generator.next()[0].shape[1:]
total_train_samples = train_generator.n
total_vdate_samples = vdate_generator.n

### Convolutional Model

In [37]:
conv_model = tf.keras.models.Sequential()
conv_model.add(Conv2D(32, kernel_size=(4, 4),
                 activation='relu',
                 input_shape=input_shape))
conv_model.add(Conv2D(32, (3, 3), activation='relu'))
conv_model.add(MaxPooling2D(pool_size=(2, 2)))
conv_model.add(Dropout(0.20))
conv_model.add(Flatten())
conv_model.add(Dense(64, activation='relu'))
conv_model.add(Dropout(0.2))
conv_model.add(Dense(2, activation='softmax'))

conv_model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
conv_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 253, 253, 32)      1568      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 251, 251, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 125, 125, 32)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 125, 125, 32)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 500000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                32000064  
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)               

In [ ]:
conv_history = conv_model.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_train_samples/batch_size),  
        epochs=n_epochs,
        verbose=1,
        validation_data=vdate_datagen,
        validation_steps=int(total_vdate_samples/batch_size),
        use_multiprocessing=False)

In [ ]:
# plot the training loss and accuracy
n_epochs_range = np.arange(0, n_epochs)
plt.style.use("ggplot")
fig, ax = plt.figure()
ax.plot(n_epochs_range, conv_history.history["loss"], label="train_loss")
ax.plot(n_epochs_range, conv_history.history["val_loss"], label="val_loss")
ax.plot(n_epochs_range, conv_history.history["acc"], label="train_acc")
ax.plot(n_epochs_range, conv_history.history["val_acc"], label="val_acc")

ax.set_title("Training Loss and Accuracy on Dataset")
ax.set_xlabel("Epoch #")
ax.set_ylabel("Loss/Accuracy")
ax.set_legend(loc="lower left")
plt.plot()

# Object Detection via Transfer Learning
This will use a pretrained model as the basis for a nn that identifies the part of an image file that contains the sidewalk object.

In [ ]:
learning_rate = 0.0001
loss = 'binary_crossentropy'

In [ ]:
pretrained_mobilenet = tf.keras.applications.MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet')
pretrained_mobilenet.trainable = False

In [ ]:
global_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(1)

mnet_model = tf.keras.Sequential([
  base_model,
  global_pooling_layer,
  prediction_layer])

mnet_model.compile(optimizer=RMSprop(lr=learning_rate),
              loss=loss,
              metrics=['accuracy'])
mnet_model.summary()

In [ ]:
mnet_history = mnet_model.fit_generator(
        train_augmented_generator, 
        steps_per_epoch=int(total_train_samples/batch_size),  
        epochs=n_epochs,
        verbose=1,
        validation_data=vdate_datagen
        validation_steps=int(total_vdate_samples/batch_size),
        use_multiprocessing=True)

# Object Segmentation

In [ ]:
model_path = "./models/yolo-tiny.h5"
input_path = "./input/test45.jpg"
output_path = "./output/newimage.jpg"

### tinyYOLOv3

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3()
detector.setModelPath(model_path)
detector.loadModel()

#returns dictionary with name and probabilities of objects detected
detection = detector.detectObjectsFromImage(input_image=input_path, output_image_path=output_path)

### ResNet